In [8]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)


def getAutoModeStatus():
    try:
        current_datetime = datetime.now()
        start_of_day = current_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = current_datetime.replace(hour=23, minute=59, second=59, microsecond=99)
        # Format datetime with 'T'
        start_of_day_str = start_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
        end_of_day_str = end_of_day.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]

        # Connect to the SQLite database
        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")

        df = pd.read_sql_query(f"SELECT start_timestamp, end_timestamp, marketprice, unit, relaynumber FROM automaterelay WHERE start_timestamp BETWEEN '{start_of_day_str}' AND '{end_of_day_str}'", conn)

        # Convert DataFrame to JSON
        json_data = df.to_json(orient='records')

        print(df)

        # Return JSON response
        return json_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"status": "Error"}), 500

    finally:
          if 'conn' in locals() and conn:
            conn.close()

getAutoModeStatus()


Current working directory: /home/pi/smart_plug/python_files
        start_timestamp        end_timestamp marketprice unit  relaynumber
0   2023-12-04T01:00:00  2023-12-04T02:00:00       88.02  kWh            1
1   2023-12-04T02:00:00  2023-12-04T03:00:00       85.38  kWh            1
2   2023-12-04T03:00:00  2023-12-04T04:00:00       85.83  kWh            1
3   2023-12-04T04:00:00  2023-12-04T05:00:00       93.59  kWh            1
4   2023-12-04T21:00:00  2023-12-04T22:00:00       100.0  kWh            1
5   2023-12-04T22:00:00  2023-12-04T23:00:00       91.76  kWh            1
6   2023-12-04T01:00:00  2023-12-04T02:00:00       88.02  kWh            2
7   2023-12-04T02:00:00  2023-12-04T03:00:00       85.38  kWh            2
8   2023-12-04T03:00:00  2023-12-04T04:00:00       85.83  kWh            2
9   2023-12-04T04:00:00  2023-12-04T05:00:00       93.59  kWh            2
10  2023-12-04T22:00:00  2023-12-04T23:00:00       91.76  kWh            2


'[{"start_timestamp":"2023-12-04T01:00:00","end_timestamp":"2023-12-04T02:00:00","marketprice":"88.02","unit":"kWh","relaynumber":1},{"start_timestamp":"2023-12-04T02:00:00","end_timestamp":"2023-12-04T03:00:00","marketprice":"85.38","unit":"kWh","relaynumber":1},{"start_timestamp":"2023-12-04T03:00:00","end_timestamp":"2023-12-04T04:00:00","marketprice":"85.83","unit":"kWh","relaynumber":1},{"start_timestamp":"2023-12-04T04:00:00","end_timestamp":"2023-12-04T05:00:00","marketprice":"93.59","unit":"kWh","relaynumber":1},{"start_timestamp":"2023-12-04T21:00:00","end_timestamp":"2023-12-04T22:00:00","marketprice":"100.0","unit":"kWh","relaynumber":1},{"start_timestamp":"2023-12-04T22:00:00","end_timestamp":"2023-12-04T23:00:00","marketprice":"91.76","unit":"kWh","relaynumber":1},{"start_timestamp":"2023-12-04T01:00:00","end_timestamp":"2023-12-04T02:00:00","marketprice":"88.02","unit":"kWh","relaynumber":2},{"start_timestamp":"2023-12-04T02:00:00","end_timestamp":"2023-12-04T03:00:00","m